In [1]:
import pandas as pd
import numpy as np
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint

#Load the Data
data = pd.read_csv("fer2013.csv")

emotion = np.array(data['emotion'])
pixels = np.array(data['pixels'])
usage = np.array(data['Usage'])

#Changing string to integer for the pixels
for i in range(0, len(pixels)):
    numbers = [int(x) for x in pixels[i].split(' ')]
    pixels[i] = numbers

#Scaling pixels from 0->255 to 0->1
for i in range(0,len(pixels)):
    for j in range(0, len(pixels[i])):
        pixels[i][j] = pixels[i][j]/255

#Splitting Data
x_train = []
y_train = []
x_valid = []
y_valid = []
x_test = []
y_test = []

for i in range(0, len(usage)):
    if (usage[i] == "Training"):
        x_train.append(pixels[i])
        y_train.append(emotion[i])
    elif (usage[i] == "PublicTest"):
        x_valid.append(pixels[i])
        y_valid.append(emotion[i])
    elif (usage[i] == "PrivateTest"):
        x_test.append(pixels[i])
        y_test.append(emotion[i])

x_train = np.array(x_train)
y_train = np.array(y_train)
x_valid = np.array(x_valid)
y_valid = np.array(y_valid)
x_test = np.array(x_test)
y_test = np.array(y_test)

#One-hot Encoding
y_train = np_utils.to_categorical(y_train, 7)
y_valid = np_utils.to_categorical(y_valid, 7)
y_test = np_utils.to_categorical(y_test, 7)

#unflatten pictures
x_train_square = []
x_valid_square = []
x_test_square = []

for i in range(len(x_train)):
    x_train_square.append(x_train[i].reshape(48,48))

for i in range(len(x_valid)):
    x_valid_square.append(x_valid[i].reshape(48,48))
    x_test_square.append(x_test[i].reshape(48,48))
    
x_train_square = np.array(x_train_square).reshape([-1,48,48,1])
x_valid_square = np.array(x_valid_square).reshape([-1,48,48,1])
x_test_square = np.array(x_test_square).reshape([-1,48,48,1])

#Data Augmentation
datagen = ImageDataGenerator(
        width_shift_range=0.1,
        height_shift_range=0.1,    
        horizontal_flip=True)
        
datagen.fit(x_train_square)

#Model Architecture
modelcnn = Sequential()
modelcnn.add(Conv2D(32, kernel_size=2, strides=1, padding='same', activation='relu', input_shape=(48,48,1)))
modelcnn.add(Conv2D(32, kernel_size=2, strides=1, padding='same', activation='relu'))
modelcnn.add(MaxPooling2D(pool_size=2))

modelcnn.add(Conv2D(64, kernel_size=2, strides=1, padding='same', activation='relu'))
modelcnn.add(Conv2D(64, kernel_size=2, strides=1, padding='same', activation='relu'))
modelcnn.add(MaxPooling2D(pool_size=2))

modelcnn.add(Conv2D(128, kernel_size=2, strides=1, padding='same', activation='relu'))
modelcnn.add(Conv2D(128, kernel_size=2, strides=1, padding='same', activation='relu'))
modelcnn.add(MaxPooling2D(pool_size=2))

modelcnn.add(Conv2D(286, kernel_size=2, strides=1, padding='same', activation='relu'))
modelcnn.add(Conv2D(286, kernel_size=2, strides=1, padding='same', activation='relu'))
modelcnn.add(GlobalAveragePooling2D())

modelcnn.add(Dense(512, activation='relu'))
modelcnn.add(Dropout(0.1))
modelcnn.add(Dense(286, activation='relu'))
modelcnn.add(Dropout(0.1))
modelcnn.add(Dense(7, activation='softmax'))
#modelcnn.summary() will show model summary

#Training Model
modelcnn.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
checkcnn = ModelCheckpoint(filepath="CNN.weights.best.hdf5", monitor='val_acc', verbose=1, save_best_only=True)
modelcnn.fit_generator(datagen.flow(x_train_square, y_train, batch_size=32),
                   steps_per_epoch=x_train_square.shape[0]//32,
                   epochs=100, verbose=2, callbacks=[checkcnn],
                   validation_data=(x_valid_square, y_valid))

Using TensorFlow backend.
C:\Users\shrfm\.conda\envs\capstone-project\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\shrfm\.conda\envs\capstone-project\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\shrfm\.conda\envs\capstone-project\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\shrfm\.conda\e

Epoch 1/100
Epoch 00000: val_acc improved from -inf to 0.24937, saving model to CNN.weights.best.hdf5
37s - loss: 1.8166 - acc: 0.2495 - val_loss: 1.7995 - val_acc: 0.2494
Epoch 2/100
Epoch 00001: val_acc improved from 0.24937 to 0.27473, saving model to CNN.weights.best.hdf5
33s - loss: 1.7940 - acc: 0.2513 - val_loss: 1.7776 - val_acc: 0.2747
Epoch 3/100
Epoch 00002: val_acc did not improve
36s - loss: 1.7734 - acc: 0.2620 - val_loss: 1.7643 - val_acc: 0.2661
Epoch 4/100
Epoch 00003: val_acc improved from 0.27473 to 0.33714, saving model to CNN.weights.best.hdf5
37s - loss: 1.7274 - acc: 0.2965 - val_loss: 1.6509 - val_acc: 0.3371
Epoch 5/100
Epoch 00004: val_acc improved from 0.33714 to 0.40680, saving model to CNN.weights.best.hdf5
62s - loss: 1.6091 - acc: 0.3572 - val_loss: 1.5329 - val_acc: 0.4068
Epoch 6/100
Epoch 00005: val_acc improved from 0.40680 to 0.45194, saving model to CNN.weights.best.hdf5
44s - loss: 1.4945 - acc: 0.4190 - val_loss: 1.4249 - val_acc: 0.4519
Epoch 7/1

Epoch 00055: val_acc did not improve
145s - loss: 0.8645 - acc: 0.6715 - val_loss: 1.0535 - val_acc: 0.6069
Epoch 57/100
Epoch 00056: val_acc did not improve
145s - loss: 0.8606 - acc: 0.6744 - val_loss: 1.0914 - val_acc: 0.6041
Epoch 58/100
Epoch 00057: val_acc did not improve
145s - loss: 0.8580 - acc: 0.6752 - val_loss: 1.0597 - val_acc: 0.6213
Epoch 59/100
Epoch 00058: val_acc did not improve
145s - loss: 0.8483 - acc: 0.6806 - val_loss: 1.0369 - val_acc: 0.6180
Epoch 60/100
Epoch 00059: val_acc did not improve
145s - loss: 0.8504 - acc: 0.6792 - val_loss: 1.0492 - val_acc: 0.6133
Epoch 61/100
Epoch 00060: val_acc did not improve
145s - loss: 0.8433 - acc: 0.6803 - val_loss: 1.0396 - val_acc: 0.6199
Epoch 62/100
Epoch 00061: val_acc did not improve
145s - loss: 0.8417 - acc: 0.6831 - val_loss: 1.0417 - val_acc: 0.6177
Epoch 63/100
Epoch 00062: val_acc did not improve
145s - loss: 0.8384 - acc: 0.6825 - val_loss: 1.0461 - val_acc: 0.6227
Epoch 64/100
Epoch 00063: val_acc did not imp

Loading Best Model and Testing Accuracy

In [2]:
modelcnn.load_weights("CNN.weights.best.hdf5")
testscore = modelcnn.evaluate(x_test_square, y_test, verbose=0)
print(100*testscore[1])

64.19615492112592
